In [23]:
import pandas as pd
import numpy as np
from BinanceCollector import BinanceCollector
from HyperBacktest import HyperBacktest
from BreakdownCalculator import BreakdownCalculator
import warnings
warnings.filterwarnings('ignore')

In [24]:
symbol = "SOLUSDT"
collector = BinanceCollector(symbol,"2022-01-01","2023-01-02","15m")
collector.start_collect()
collector.df.columns = ["Open Time","Open","High","Low","Close","Volume","Close Time",
             "Quote asset volumne","Total Trades","Buyer Base Volume",
             "Taker buy quote asset volume","Ignore"]
df = collector.df
breakdown = BinanceCollector(symbol,"2022-01-01","2023-01-02","1m")
breakdown.start_collect()
breakdown.df.columns = ["Open Time","Open","High","Low","Close","Volume","Close Time",
             "Quote asset volumne","Total Trades","Buyer Base Volume",
             "Taker buy quote asset volume","Ignore"]
breakdown_df = breakdown.df
calculator = BreakdownCalculator(breakdown_df)

In [25]:
df["Open Time"] = pd.to_datetime(df["Open Time"],unit="ms")
df["Close Time"] = pd.to_datetime(df["Close Time"],unit="ms")
df["Returns"] =  0

In [28]:
for i in range(1,200):
    for j in range(i+1,200):
        df["EMA_signal"] = df["Close"].ewm(span=i,adjust=False).mean()
        df["EMA_lead"] = df["Close"].ewm(span=j,adjust=False).mean()

        df["Buy"] = np.where( (df["EMA_signal"] > df["EMA_lead"]) & (df['EMA_signal'].shift(1) <= df["EMA_lead"].shift(1) ),1 ,0 )
        df["Sell"] = np.where( (df["EMA_signal"] < df["EMA_lead"] ) & ( df["EMA_signal"].shift(1) >= df["EMA_lead"].shift(1) ),1 ,0 )
        backtest = HyperBacktest(4,-2,calculator,df)
        backtest.run()
        results = backtest.df
        # results["Returns"].cumsum().plot(figsize=(10,5))
        print(f"EMA Signal: {i} EMA Lead: {j} Returns: {results['Returns'].sum()}")
        print("---------------------------------")

KeyError: -1